In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e18/sample_submission.csv
/kaggle/input/playground-series-s3e18/train.csv
/kaggle/input/playground-series-s3e18/test.csv
/kaggle/input/ec-mixed-class/mixed_fcfp.csv
/kaggle/input/ec-mixed-class/mixed_desc.csv
/kaggle/input/ec-mixed-class/mixed_ecfp.csv


In [2]:
train_data=pd.read_csv('/kaggle/input/playground-series-s3e18/train.csv')
test_data=pd.read_csv('/kaggle/input/playground-series-s3e18/test.csv')
sample=pd.read_csv('/kaggle/input/playground-series-s3e18/sample_submission.csv')

In [3]:
train_data.head()

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
0,0,323.390782,9.879918,5.875576,5.875576,4.304757,4.304757,2.754513,1.749203,0.000000,...,4.794537,35.527357,0,0,1,1,0,0,0,0
1,1,273.723798,7.259037,4.441467,5.834958,3.285046,4.485235,2.201375,1.289775,45.135471,...,13.825658,44.707310,0,0,0,1,1,0,0,0
2,2,521.643822,10.911303,8.527859,11.050864,6.665291,9.519706,5.824822,1.770579,15.645394,...,17.964475,45.660120,0,0,1,1,0,0,1,0
3,3,567.431166,12.453343,7.089119,12.833709,6.478023,10.978151,7.914542,3.067181,95.639554,...,31.961948,87.509997,0,0,1,1,0,0,0,0
4,4,112.770735,4.414719,2.866236,2.866236,1.875634,1.875634,1.036450,0.727664,17.980451,...,9.589074,33.333333,2,2,1,0,1,1,1,0


In [4]:
train_data.isna().sum().sum()

0

In [5]:
test_data.shape

(9893, 32)

In [6]:
import pandas as pd

# Get the column names from each dataset
train_columns = set(train_data.columns)
test_columns = set(test_data.columns)

# Find the uncommon columns
uncommon_columns = train_columns.symmetric_difference(test_columns)

# Print the uncommon columns
print("Uncommon columns between train_data and test_data:")
for column in uncommon_columns:
    print(column)

Uncommon columns between train_data and test_data:
EC6
EC4
EC3
EC2
EC1
EC5


In [7]:
train_data.columns

Index(['id', 'BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3v',
       'Chi4n', 'EState_VSA1', 'EState_VSA2', 'ExactMolWt', 'FpDensityMorgan1',
       'FpDensityMorgan2', 'FpDensityMorgan3', 'HallKierAlpha',
       'HeavyAtomMolWt', 'Kappa3', 'MaxAbsEStateIndex', 'MinEStateIndex',
       'NumHeteroatoms', 'PEOE_VSA10', 'PEOE_VSA14', 'PEOE_VSA6', 'PEOE_VSA7',
       'PEOE_VSA8', 'SMR_VSA10', 'SMR_VSA5', 'SlogP_VSA3', 'VSA_EState9',
       'fr_COO', 'fr_COO2', 'EC1', 'EC2', 'EC3', 'EC4', 'EC5', 'EC6'],
      dtype='object')

finding the number of similar test_data columns

In [8]:
test_data.columns

Index(['id', 'BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3v',
       'Chi4n', 'EState_VSA1', 'EState_VSA2', 'ExactMolWt', 'FpDensityMorgan1',
       'FpDensityMorgan2', 'FpDensityMorgan3', 'HallKierAlpha',
       'HeavyAtomMolWt', 'Kappa3', 'MaxAbsEStateIndex', 'MinEStateIndex',
       'NumHeteroatoms', 'PEOE_VSA10', 'PEOE_VSA14', 'PEOE_VSA6', 'PEOE_VSA7',
       'PEOE_VSA8', 'SMR_VSA10', 'SMR_VSA5', 'SlogP_VSA3', 'VSA_EState9',
       'fr_COO', 'fr_COO2'],
      dtype='object')

finding the number of unique values of the target columns to understand how to classify

In [9]:
train_data['EC2'].unique()

array([1, 0])

therefore the values of EC1 and EC2 are unique

storing the training and testing data as needed

In [10]:
# Extract column names for test_1 dataset
test_columns = test_data.columns[1:]

# Create test_1 dataset from test_data
test_1 = test_data[test_columns]

# Extract column names for train_1 dataset
train_columns = [column for column in train_data.columns if column in test_columns]

# Create train_1 dataset from train_data
train_1 = train_data[train_columns]

In [11]:
train_1.isnull().sum().sum()

0

basic rf modelling to kick it off

In [12]:
y_train_1=train_data['EC1']
y_train_2=train_data['EC2']

fitting and predicting the first category column

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Create an object of RandomForestClassifier
model1= RandomForestClassifier()

model1.fit(train_1,y_train_1)
y_pred_1=model1.predict(test_1)

doing the same but with grid search instead

In [14]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV

# # Create an object of RandomForestClassifier
# rf_classifier = RandomForestClassifier()

# # Define the parameter grid
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [None, 5, 10],
#     'min_samples_split': [2, 4, 8],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Create GridSearchCV object
# grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5)

# # Fit the GridSearchCV object to the data
# grid_search.fit(train_1, y_train_1)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print("Best parameters:", best_params)
# print("Best score:", best_score)


In [15]:
from sklearn.ensemble import RandomForestClassifier

# Define the best parameters
best_params = {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}

# Create the RandomForestClassifier model with the best parameters
model = RandomForestClassifier(**best_params)
model.fit(train_1,y_train_1)
y_pred_1=model.predict(test_1)

Best parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best score: 0.7073724718534609

In [16]:
test_1.shape[0]==len(y_pred_1)

True

doing grid search for the 2nd target column as well

In [17]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV

# # Create an object of RandomForestClassifier
# rf_classifier = RandomForestClassifier()

# # Define the parameter grid
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [None, 5, 10],
#     'min_samples_split': [2, 4, 8],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Create GridSearchCV object
# grid_search_2= GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5)

# # Fit the GridSearchCV object to the data
# grid_search.fit(train_1, y_train_2)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print("Best parameters:", best_params)
# print("Best score:", best_score)


Best parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Best score: 0.7990295314951437

fitting the model as required

In [18]:
model2 = RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=50)
model2.fit(train_1, y_train_2)
y_pred_2 = model2.predict(test_1)

In [19]:
len(y_pred_2)==test_1.shape[0]

True

creating a submission file

first creating a pandas dataframe as needed

In [20]:
# Create a DataFrame with 'id' column from test_1
df = pd.DataFrame({'id': test_data['id']})

# Add 'EC1' column containing the array y_train_1
df['EC1'] = pd.Series(y_pred_1)

# Add 'EC2' column containing the array y_train_2
df['EC2'] = pd.Series(y_pred_2)

In [21]:
df.shape

(9893, 3)

In [22]:
df.columns

Index(['id', 'EC1', 'EC2'], dtype='object')

In [23]:
sample.shape

(9893, 3)

submitting the df dataset as needed

In [24]:
df.to_csv('submission.csv',index=False)